In [47]:
import os

from bids import BIDSLayout

from glob import glob

from nipype.interfaces.io import BIDSDataGrabber
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.utility import Function

In [48]:
bids_dir = os.path.join('/fetaldata')
output_dir = os.path.join('/fetaldata','derivatives','mialsrtk')
mask_dir = os.path.join('/fetaldata','derivatives','manual_masks')

subject = '01'

layout = BIDSLayout(bids_dir,derivatives=mask_dir)
print(layout)

import shutil
shutil.rmtree(output_dir)

BIDS Layout: .../fetaldata | Subjects: 1 | Sessions: 0 | Runs: 6


In [49]:
import subprocess
def run(self, command, env={}, cwd=os.getcwd()):
    merged_env = os.environ
    merged_env.update(env)
    process = subprocess.run(command, shell=True,
                               env=merged_env, cwd=cwd, capture_output=True)
    return process

In [50]:
from traits.api import *

import nibabel as nib

from nipype.utils.filemanip import split_filename
from nipype.interfaces.base import traits, isdefined, CommandLine, CommandLineInputSpec,\
    TraitedSpec, File, InputMultiPath, OutputMultiPath, BaseInterface, BaseInterfaceInputSpec

from nipype.interfaces.mixins import reporting



In [52]:
class MialsrtkTVSuperResolutionInputSpec(BaseInterfaceInputSpec):
    bids_dir = Directory(desc='BIDS root directory',mandatory=True,exists=True)
    in_file = File(desc='Input image',mandatory=True)
    in_mask = File(desc='Input mask',mandatory=False)
    out_postfix = traits.Str("_SR", usedefault=True)
    debluring = traits.Bool(False, usedefault=True)

class MialsrtkTVSuperResolutionOutputSpec(TraitedSpec):
    out_file = File(desc='SR image')

class MialsrtkTVSuperResolution(BaseInterface):

    input_spec = MialsrtkTVSuperResolutionInputSpec
    output_spec = MialsrtkTVSuperResolutionOutputSpec
    
    def _run_interface(self, runtime): 
        _, name, ext = split_filename(os.path.abspath(self.inputs.in_file))
        out_file = os.path.join(os.getcwd().replace(self.inputs.bids_dir,'/fetaldata'), ''.join((name, self.inputs.out_postfix, ext)))
        
        if self.inputs.in_mask:
            cmd = 'mialsrtkTVSuperResolution -i "{}" -o "{}" -m "{}" --debluring {}'.format(self.inputs.in_file,out_file,self.inputs.in_mask,self.inputs.debluring)
        else:
            cmd = 'mialsrtkTVSuperResolution -i "{}" -o "{}" --debluring {}'.format(self.inputs.in_file,out_file,self.inputs.debluring)
        
        try:
            print('... cmd: {}'.format(cmd))
            run(self, cmd, env={}, cwd=os.path.abspath(self.inputs.bids_dir))
        except:
            print('Failed')
        return runtime

    def _list_outputs(self):
        outputs = self._outputs().get()
        _, name, ext = split_filename(os.path.abspath(self.inputs.in_file))
        outputs['out_file'] = os.path.join(os.getcwd().replace(self.inputs.bids_dir,'/fetaldata'), ''.join((name, self.inputs.out_postfix, ext)))
        return outputs
    
class MultipleMialsrtkTVSuperResolutionInputSpec(BaseInterfaceInputSpec):
    bids_dir = Directory(desc='BIDS root directory',mandatory=True,exists=True)
    input_images = InputMultiPath(File(desc='files to be SR', mandatory = True))
    input_masks = InputMultiPath(File(desc='mask of files to be SR', mandatory = False))
    debluring = traits.Bool(False)
    out_postfix = traits.Str("_SR", usedefault=True)
    
class MultipleMialsrtkTVSuperResolutionOutputSpec(TraitedSpec):
    output_images = OutputMultiPath(File())

class MultipleMialsrtkTVSuperResolution(BaseInterface):
    input_spec = MultipleMialsrtkTVSuperResolutionInputSpec
    output_spec = MultipleMialsrtkTVSuperResolutionOutputSpec

    def _run_interface(self, runtime):
        if len(self.inputs.input_images)>0:
            for input_image, input_mask in zip(self.inputs.input_images,self.inputs.input_masks):
                ax = MialsrtkTVSuperResolution(bids_dir = self.inputs.bids_dir, in_file = input_image, in_mask = input_mask, out_postfix=self.inputs.out_postfix, debluring = self.inputs.debluring)
                ax.run()
        else:
            for input_image in self.inputs.input_images:
                ax = MialsrtkTVSuperResolution(bids_dir = self.inputs.bids_dir, in_file = input_image, out_postfix=self.inputs.out_postfix, debluring = self.inputs.debluring)
                ax.run()
        return runtime

    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs['output_images'] = glob(os.path.abspath("*.nii.gz"))
        return outputs

In [53]:
#Check if mandatory derivatives dataset_description.json exists in derivatives/mialsrtk.
# If not, it is created before running the workflow, otherwise BIDSDataGrabber is not happy and raises an error. 

mialsrtk_dataset_description_json = os.path.join(output_dir,'dataset_description.json')

print('Check for {}'.format(mialsrtk_dataset_description_json))
if not os.access(mialsrtk_dataset_description_json, os.R_OK):
    import json
    data = {'PipelineDescription':{'Name': 'MIAL Super-Resolution ToolKit', 
                                'Version': 'v2.0.0-beta', 
                                'CodeURL': 'https://github.com/sebastientourbier/mialsuperresolutiontoolkit'
                                  },
            'Name': 'MIAL Super-Resolution ToolKit',
            'BIDSVersion': '1.2.0'
           }
    os.makedirs(output_dir)
    with open(mialsrtk_dataset_description_json, 'w+') as outfile:
        json.dump(data, outfile, indent=4)
    print('File {} was created'.format(mialsrtk_dataset_description_json))
else:
    print('File {} already exists'.format(mialsrtk_dataset_description_json))
    

wf = Workflow(name="image_SR",base_dir=os.path.join(output_dir,'sub-{}'.format(subject),'nipype'))

bg = Node(interface=BIDSDataGrabber(infields = ['subject']),name='bids_grabber')
bg.inputs.base_dir = bids_dir
bg.inputs.subject = subject
bg.inputs.index_derivatives = True
bg.inputs.raise_on_empty = False
bg.inputs.output_query = {'T2ws': dict(suffix='T2w',datatype='anat',extensions=[".nii",".nii.gz"]),
                          'masks': dict(suffix='mask',datatype='anat',extensions=[".nii",".nii.gz"])}
imgSR = Node(interface = MultipleMialsrtkTVSuperResolution(),name='image_SR')
imgSR.inputs.bids_dir = bids_dir
imgSR.inputs.debluring = False

wf.connect(bg, "T2ws", imgSR, "input_images")
wf.connect(bg, "masks", imgSR, "input_masks")

res = wf.run()

Check for /fetaldata/derivatives/mialsrtk/dataset_description.json
File /fetaldata/derivatives/mialsrtk/dataset_description.json was created
191023-17:39:25,45 nipype.workflow INFO:
	 Workflow image_SR settings: ['check', 'execution', 'logging', 'monitoring']
191023-17:39:25,50 nipype.workflow INFO:
	 Running serially.
191023-17:39:25,51 nipype.workflow INFO:
	 [Node] Setting-up "image_SR.bids_grabber" in "/fetaldata/derivatives/mialsrtk/sub-01/nipype/image_SR/bids_grabber".
191023-17:39:25,55 nipype.workflow INFO:
	 [Node] Running "bids_grabber" ("nipype.interfaces.io.BIDSDataGrabber")


/opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/bids/layout/layout.py:690: UserWarning: In pybids 0.9.0, the 'extensions' filter was deprecated in favor of 'extension'. The former will stop working in 0.11.0.
  warnings.warn("In pybids 0.9.0, the 'extensions' filter was "


191023-17:39:25,980 nipype.workflow INFO:
	 [Node] Finished "image_SR.bids_grabber".
191023-17:39:25,981 nipype.workflow INFO:
	 [Node] Setting-up "image_SR.image_SR" in "/fetaldata/derivatives/mialsrtk/sub-01/nipype/image_SR/image_SR".
191023-17:39:25,985 nipype.workflow INFO:
	 [Node] Running "image_SR" ("__main__.MultipleMialsrtkTVSuperResolution")
... cmd: mialsrtkTVSuperResolution -i "/fetaldata/sub-01/anat/sub-01_run-1_T2w.nii.gz" -o "/fetaldata/derivatives/mialsrtk/sub-01/nipype/image_SR/image_SR/sub-01_run-1_T2w_SR.nii.gz" -m "/fetaldata/derivatives/manual_masks/sub-01/anat/sub-01_run-1_T2w_desc-brain_mask.nii.gz" --debluring False
... cmd: mialsrtkTVSuperResolution -i "/fetaldata/sub-01/anat/sub-01_run-2_T2w.nii.gz" -o "/fetaldata/derivatives/mialsrtk/sub-01/nipype/image_SR/image_SR/sub-01_run-2_T2w_SR.nii.gz" -m "/fetaldata/derivatives/manual_masks/sub-01/anat/sub-01_run-2_T2w_desc-brain_mask.nii.gz" --debluring False
... cmd: mialsrtkTVSuperResolution -i "/fetaldata/sub-01/a

___

___